### Importing the Libaries

In [1]:
import numpy as np
import pandas as pd
from pulp import *
import math
import itertools

### Defining the Data

In [175]:
def define_data():
    df = {}
    
    df['weight'] = [200] *2 + [80] * 20 + [50] * 15 + [30] * 12 + [25] * 20
    df['boxes'] = [i for i in range(len(df['weight']))]
    
    df['length'] = [30] *2 + [15] * 20 + [10] * 15 + [8] * 12 + [5] * 20
    df['breadth'] = [25]*2 + [15] * 20 + [10] * 15 + [8] * 12 + [5] * 20
    df['height'] = [25] * 2 + [15] * 20 + [10] * 15 + [8] * 12 + [5] * 20
    df['volume'] = list(np.array(df['length']) * np.array(df['breadth']) * np.array(df['height']))
#     df['truck_weight'] = 1000
    df['total_truck_fleet'] = [i for i in range(len(df['weight']))]
    return df

In [178]:
data = define_data()
pd.DataFrame(data).head()

,weight,boxes,length,breadth,height,volume,total_truck_fleet
0,200,0,30,25,25,18750,0
1,200,1,30,25,25,18750,1
2,80,2,15,15,15,3375,2
3,80,3,15,15,15,3375,3
4,80,4,15,15,15,3375,4


In [100]:
df = pd.DataFrame()
df['weight'] =[50] * 15 + [20]* 15
df['boxes'] = [i for i in range(len(df['weight']))]

df['length'] = [60] * 15 + [40] * 15
df['breadth'] = [40] * 15 + [35] * 15
df['height'] = [30] * 15 + [35] * 15
df['volume'] = list(np.array(df['length']) * np.array(df['breadth']) * np.array(df['height']))
#     df['truck_weight'] = 1000
df['total_truck_fleet'] = [i for i in range(len(df['weight']))]

In [101]:
data = df.copy()
data

,weight,boxes,length,breadth,height,volume,total_truck_fleet
0,50,0,60,40,30,72000,0
1,50,1,60,40,30,72000,1
2,50,2,60,40,30,72000,2
3,50,3,60,40,30,72000,3
4,50,4,60,40,30,72000,4
5,50,5,60,40,30,72000,5
6,50,6,60,40,30,72000,6
7,50,7,60,40,30,72000,7
8,50,8,60,40,30,72000,8
9,50,9,60,40,30,72000,9


### Defining the Objectve Variable

### Defining the dimensions of the truck and range of co-ordinates

In [102]:
length_of_truck = 250
breadth_of_truck = 150
height_of_truck = 100

min_height = 50
min_breadth = 30
min_length = 40
volume_of_truck = length_of_truck * breadth_of_truck * height_of_truck
weight_of_truck = 10000
x_axis = math.ceil(length_of_truck / min_length)
y_axis = math.ceil(breadth_of_truck / min_breadth)
z_axis = math.ceil(height_of_truck / min_height)

### Defining the Objective Function and Base Constraints

In [52]:
volume_of_truck

3750000

In [53]:
y_ax

4

#### boxes_loaded variable stores the binary info whether a box i is loaded in truck j

In [103]:
boxes_loaded = LpVariable.dicts("AtLocation",
[(i, j) for i in data['boxes']
for j in data['total_truck_fleet']],
0, 1, LpBinary)

### Creating a dictionary of dictionaries

Each truck stores a dictionary of info (each coordinate of the truck stores this info):
- whether a box is loaded in that coordinate
- if yes
  - length of box loaded
  - breadth of box loaded
  - height of box loaded
  - volume of box loaded

In [104]:
dict_truck_info = {}
for tr in data['total_truck_fleet']:
    for i in range(x_axis):
        for j in range(y_axis):
            for k in range(1, z_axis):
                dict_truck_info[(tr, i, j ,k)] = {'box_loaded': LpVariable("loaded", 0, 1, cat = 'Binary'),
                                                 'length_loaded': LpVariable("length", 0, length_of_truck, cat = 'Continuous'),
                                                 'breadth_loaded': LpVariable("breadth", 0, breadth_of_truck, cat = 'Continuous'),
                                                 'height_loaded': LpVariable("height", 0, height_of_truck, cat = 'Continuous'),
                                                 'volume_loaded': LpVariable("volume", 0, volume_of_truck, cat = 'Continuous'),
                                                 'surface_area': LpVariable(f"{tr}_surface_area_{(i, j, k)}", 0, volume_of_truck, cat = 'Continuous')}

In [105]:
# dict_truck_info

In [106]:
def dict_update(dict_truck_info, truck_num, boxes_data, box_num, axes, dimensions, update_dimensions):
    
    x_ax = axes[0]
    y_ax = axes[1]
    z_ax = axes[2]
    
    remaining_length = dimensions[0]
    remaining_breadth = dimensions[1]
    remaining_height = dimensions[2]
    
    dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['box_loaded'] = 1
    dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['length_loaded'] = data['length'][box_num]
    dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['breadth_loaded'] = data['breadth'][box_num]
    dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['heigth_loaded'] = data['height'][box_num]
    
    dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['surface_area'] = data['height'][box_num] * data['length'][box_num]
    
    if update_dimensions[0] is True:
        remaining_length -= dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['length_loaded']
    
    if update_dimensions[1] is True:
        remaining_breadth -= dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['breadth_loaded']
    
    if update_dimensions[2] is True:
        remaining_height -= dict_truck_info[(truck_num, x_ax, y_ax, z_ax)]['height_loaded']
    
    return dict_truck_info, remaining_length, remaining_breadth, remaining_height

In [107]:
list_of_axes = [[x for x in range(x_axis)] , [y for y in range(y_axis)] , [z for z in range(z_axis)]]
list_coordinates = list(itertools.product(*list_of_axes))


In [108]:
remaining_weight = weight_of_truck# - data['weight'][i]
remaining_volume = volume_of_truck # - data['vlume'][i]

### Simulation For Placement of boxes inside truck

In [109]:
for j in data['total_truck_fleet']:
    remaining_length = length_of_truck
    remaining_breadth = breadth_of_truck
    remaining_height = height_of_truck
    remaining_weight = weight_of_truck
    remaining_volume = volume_of_truck
    
    for i in data['boxes']:
#         if boxes_loaded[(i, j)].varValue:
#             if boxes_loaded[(i, j)].varValue > 0:
        for val in list_coordinates:
            x_ax = val[0]; y_ax = val[1]; z_ax = val[2]

        while(remaining_volume >0 and remaining_weight > 0):
            if x_ax >= x_axis and y_ax >= y_axis and z_ax >= z_axis:
                boxes_loaded[(i, j)].varValue = 0
                break
            elif x_ax >= x_axis and y_ax >= y_axis and z_ax < z_axis:
                x_ax = val[0]; y_ax = val[1]; z_ax += 1
                continue
            elif x_ax >= x_axis and y_ax < y_axis and z_ax < z_axis:
                x_ax = val[0]; y_ax += 1
                continue

            elif y_ax >= y_axis:
                x_ax = val[0]; y_ax = val[1]; z_ax += 1
                continue

            else:
                boxes_loaded[(i, j)].varValue = 0
                break


            if dict_truck_info[(j, x_ax, y_ax, z_ax)]['box_loaded'] == 1:
                if remaining_length >0 and remaining_breadth > 0 and remaining_height > 0:
                    x_ax += 1
                    continue
                elif remaining_length <= 0 and remaining_breadth > 0 and reamining_height > 0:
                    x_ax = val[0]; y_ax += 1
                    continue
                elif remaining_length <= 0 and remaining_breadth <= 0 and reamining_height > 0:
                    x_ax = val[0]; y_ax = val[1]; z_ax += 1
                    continue
                else:
                    break


            elif dict_truck_info[(j, x_ax, y_ax, z_ax)]['box_loaded'] == 0:
                if remaining_length >0 and remaining_breadth > 0 and reamining_height > 0:
                    if z_ax == 0:
                        dict_truck_info,\
                        remaining_length,\
                        remaining_breadth,\
                        remaining_height = dict_update(dict_truck_info = dict_truck_info,
                                                     truck_num = j,
                                                     boxes_data = data,
                                                     box_num = i,
                                                     axes = [a_ax, y_ax, z_ax], 
                                                     dimensions = [remaining_length, remaining_breadth, remaining_height],
                                                     update_dimensions = [False, True, False])
                        boxes_loaded[(i, j)].varValue = 1
                    else:
                        if dict_truck_info[(j, x_ax, y_ax, z_ax-1)]['volume_loaded'] > data['volume'][i]:
                            dict_truck_info,\
                            remaining_length,\
                            remaining_breadth,\
                            remaining_height = dict_update(dict_truck_info = dict_truck_info,
                                                         truck_num = j,
                                                         boxes_data = data,
                                                         box_num = i,
                                                         axes = [a_ax, y_ax, z_ax], 
                                                         dimensions = [remaining_length, remaining_breadth, remaining_height],
                                                         update_dimensions = [False, True, False])
                            boxes_loaded[(i, j)].varValue = 1
                        else:
                            x_ax += 1
                            continue

                elif remaining_length <= 0 and remaining_breadth > 0 and reamining_height > 0:
                    x_ax = val[0]; y_ax += 1
                    continue
                elif remaining_length <= 0 and remaining_breadth <= 0 and reamining_height > 0:
                    x_ax = val[0]; y_ax = val[1]; z_ax += 1
                    continue
                else:
                    boxes_loaded[(i, j)].varValue = 0
                    break



In [110]:
y = [LpVariable("t{0}".format(i+1), cat=LpBinary) for i in data['total_truck_fleet']]

In [111]:
prob = LpProblem("truckLoading", LpMinimize)



In [112]:
for truck in data['total_truck_fleet']:
    for y_ in range(1, y_ax):
#                 list_of_vars.append(dict_truck_info[truck, x_, y_, z_]['surface_area'].varValue)
        prob += lpSum([dict_truck_info[truck, x_, y_, z_]['surface_area'] for x_ in range(1, x_ax) for z_ in range(1, z_ax)]) <= breadth_of_truck * height_of_truck

In [113]:
prob += lpSum([y[i] for i in data['total_truck_fleet']])

In [114]:
#For each box, it can be placed in only one truck

for i in data['boxes']:
    prob += lpSum(boxes_loaded[(i, j)] for j in data['total_truck_fleet']) == 1

In [115]:
#for each truck,
#    summation of weight of all boxes should be less than total weight truck can carry
#    summation of volume of all boxes should be less than volume of the truck


for j in data['total_truck_fleet']:
    prob += lpSum(boxes_loaded[(i, j)] * data['weight'][i] for i in data['boxes']) <= weight_of_truck * y[j]
    prob += lpSum(boxes_loaded[(i, j)] * data['volume'][i] for i in data['boxes']) <= volume_of_truck * y[j]


In [88]:
prob.solve()

1

In [85]:
LpStatus[prob.status]

'Optimal'

In [190]:
volume_of_truck

3600000

In [191]:
weight_of_truck

800

In [121]:
prob.solve()
TOL = 0.00001
truck_num = 0
total_trucks_used = 0
for j in data['total_truck_fleet']:
    if y[j].varValue > TOL:
        total_trucks_used += 1
print(f"Total Trucks Used: {total_trucks_used}")
print("\n")
print("="* 100)
for j in data['total_truck_fleet']:
    
    if y[j].varValue > TOL:
        print(f"Truck Number: {truck_num+1}")
        print("=" * 100)
        total_weight_loaded = 0
        total_num_boxes = 0
        total_volume_loaded = 0
        for i in data['boxes']:
            if boxes_loaded[(i, j)].varValue > TOL:
                total_num_boxes += 1
                total_weight_loaded += data['weight'][i]
                total_volume_loaded += data['volume'][i]
        print(f'Total Boxes Loaded: {total_num_boxes}')
        print(f"Total Weight Loaded: {total_weight_loaded}")
        print(f"Total Volume Loaded: {total_volume_loaded}")
        print("-" * 80)
        
        for i in data['boxes']:
            if boxes_loaded[(i, j)].varValue > TOL:
                
                box_num = data['boxes'][i]
                len_ = data['length'][i]
                br_ = data['breadth'][i]
                hei_ = data['height'][i]
                wt_ = data['weight'][i]
                print(f"\t Box Number: {box_num}", f"\t Dimensions of Box: {len_, br_, hei_}", f"\t Weight of Box: {wt_}", sep = "\n")
                print("-" * 80)
#         print ("Truck ", j, " holds ", \
#         [i for i in data['boxes']
#         if boxes_loaded[(i, j)].varValue > TOL])
        print("=" * 100)
        truck_num+= 1

Total Trucks Used: 1


Truck Number: 1
Total Boxes Loaded: 30
Total Weight Loaded: 1050
Total Volume Loaded: 1815000
--------------------------------------------------------------------------------
	 Box Number: 0
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 1
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 2
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 3
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 4
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 5
	 Dimensions of Box: (60, 40, 3

In [246]:
y

array([1*t1 + 2489460, 1*t2 + 2489460, 1*t3 + 2489460, 1*t4 + 2489460,
       1*t5 + 2489460, 1*t6 + 2489460, 1*t7 + 2489460, 1*t8 + 2489460,
       1*t9 + 2489460, 1*t10 + 2489460, 1*t11 + 2489460, 1*t12 + 2489460],
      dtype=object)

In [122]:
prob.solve()
TOL = 0.00001
truck_num = 0
total_trucks_used = 0
for j in data['total_truck_fleet']:
    if y[j].varValue > TOL:
        
        total_trucks_used += 1
        
print(f"Total Trucks Used: {total_trucks_used}")
print("\n")
print("="* 100)
for j in data['total_truck_fleet']:
    
    if y[j].varValue > TOL:
        print(f"Truck Number: {truck_num+1}")
        print("=" * 100)
        
        
        
        total_weight_loaded = 0
        total_num_boxes = 0
        total_volume_loaded = 0
        for i in data['boxes']:
            if boxes_loaded[(i, j)].varValue > TOL:
                
                total_num_boxes += 1
                total_weight_loaded += data['weight'][i]
                total_volume_loaded += data['volume'][i]
        print(f'Total Boxes Loaded: {total_num_boxes}')
        print(f"Total Weight Loaded: {total_weight_loaded}")
        print(f"Total Volume Loaded: {total_volume_loaded}")
        print("-" * 80)
        x_ = 0
        y_ = 0
        z_ = 0
        dict_ = {(0, 0, 0): {'loaded': 0,
                             'length': 0,
                             'width': 0, 
                             'height': 0}}
        df_output = pd.DataFrame()
        df_output['truck_num'] = truck_num + 1
        for i in data['boxes']:
            if boxes_loaded[(i, j)].varValue > TOL:
                
                df_output['Truck_Number'] = truck_num + 1
                df_output['Box Number'] = data['boxes'].iloc[i]
                df_output['L'] = data['length'].iloc[i]
                df_output['W'] = data['breadth'].iloc[i]
                df_output['H'] = data['height'].iloc[i]
                
                box_num = data['boxes'][i]
                len_ = data['length'][i]
                br_ = data['breadth'][i]
                hei_ = data['height'][i]
                wt_ = data['weight'][i]
                
                while total_num_boxes != 0:
                    if dict_[(x_, y_, z_)]['loaded'] == 0 and x_ <= length_of_truck and y_ <= breadth_of_truck and z_ <= height_of_truck:
                        dict_[(x_, y_, z_)]['loaded'] = 1
                        dict_[(x_, y_, z_)]['length'] = len_
                        dict_[(x_, y_, z_)]['width'] = br_
                        dict_[(x_, y_, z_)]['height'] = hei_
                        

                        x_ += len_
                        total_num_boxes -= 1
                        if x_ <= length_of_truck:
                            dict_[(x_, y_, z_)] ={'loaded': 0,
                                 'length': 0,
                                 'width': 0, 
                                 'height': 0}
                        break

                    elif dict_[(x_, y_, z_)]['loaded'] == 1:
                        x_ += len_
                        dict_[(x_, y_, z_)] ={'loaded': 0,
                             'length': 0,
                             'width': 0, 
                             'height': 0}
                        continue
                    
                    elif x_ > length_of_truck:
                        x_ = 0
                        inc_value = dict_[(0, y_, z_)]['width']
                        y_ += inc_value
                        dict_[(x_, y_, z_)] ={'loaded': 0,
                             'length': 0,
                             'width': 0, 
                             'height': 0}
                        continue
                        
                    
                    elif x_ <= length_of_truck and y_ > breadth_of_truck and z_ <= height_of_truck:
                        x_ += len_
                        
                    elif x_ > length_of_truck and y_ > breadth_of_truck:
                        x_ = 0
                        y_ = 0
                        z_ += dict_[(0, 0, 0)]['height']
                        
                    
                        
                    
                
                print(f"\t Box Number: {box_num}", f"\t Dimensions of Box: {len_, br_, hei_}", f"\t Weight of Box: {wt_}", sep = "\n")
                print("-" * 80)
#         print ("Truck ", j, " holds ", \
#         [i for i in data['boxes']
#         if boxes_loaded[(i, j)].varValue > TOL])
        print("=" * 100)
        truck_num+= 1

Total Trucks Used: 1


Truck Number: 1
Total Boxes Loaded: 30
Total Weight Loaded: 1050
Total Volume Loaded: 1815000
--------------------------------------------------------------------------------
	 Box Number: 0
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 1
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 2
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 3
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------
	 Box Number: 4
	 Dimensions of Box: (60, 40, 30)
	 Weight of Box: 50
--------------------------------------------------------------------------------


KeyError: (300, 0, 0)

In [118]:
dict_

{(0, 0, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (60, 0, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (120, 0, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (180, 0, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (240, 0, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (300, 0, 0): {'loaded': 0, 'length': 0, 'width': 0, 'height': 0},
 (0, 40, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (60, 40, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (120, 40, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (180, 40, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (240, 40, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (300, 40, 0): {'loaded': 0, 'length': 0, 'width': 0, 'height': 0},
 (0, 80, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (60, 80, 0): {'loaded': 1, 'length': 60, 'width': 40, 'height': 30},
 (120, 80, 0): {'loaded': 1

In [285]:
pd.
for index, keys in enumerate(dict_.keys()):
    

TypeError: list indices must be integers or slices, not str

In [92]:
df_output = pd.DataFrame(index = [i for i in range(len(dict_.keys()))], columns = ['truck_number', 'box_number', 'X', 'Y', 'Z', 'L', 'W', 'H'])
for index, keys in enumerate(dict_.keys()):
    if dict_[keys]['loaded'] == 1:
        df_output['truck_number'].loc[index] = index
        df_output['box_number'].loc[index] = index
        df_output['X'].loc[index] = keys[0]
        df_output['Y'].loc[index] = keys[1]
        df_output['Z'].loc[index] = keys[2]
        df_output['L'].loc[index] = dict_[keys]['length']
        df_output['W'].loc[index] = dict_[keys]['width']
        df_output['H'].loc[index] = dict_[keys]['height']

In [93]:
df_output.dropna(inplace = True, axis = 'rows')
df_output.reset_index(inplace = True, drop = True)
df_output = df_output[df_output.columns.tolist()[2:]]
df_output['Pack'] = df_output.index.tolist()
df_output

,X,Y,Z,L,W,H,Pack
0,0,0,0,30,20,15,0
1,30,0,0,30,20,15,1
2,60,0,0,30,20,15,2
3,90,0,0,30,20,15,3
4,120,0,0,30,20,15,4
5,150,0,0,30,20,15,5
6,180,0,0,30,20,15,6
7,210,0,0,30,20,15,7
8,240,0,0,30,20,15,8
9,0,20,0,30,20,15,9


In [94]:
df_output['Pack'] = list(map(lambda x: "pack_" + str(x+1), df_output['Pack']))

In [97]:
df_output['Pull_forward_flag'] = 0

In [98]:
df_output

,X,Y,Z,L,W,H,Pack,Pull_forward_flag
0,0,0,0,30,20,15,pack_1,0
1,30,0,0,30,20,15,pack_2,0
2,60,0,0,30,20,15,pack_3,0
3,90,0,0,30,20,15,pack_4,0
4,120,0,0,30,20,15,pack_5,0
5,150,0,0,30,20,15,pack_6,0
6,180,0,0,30,20,15,pack_7,0
7,210,0,0,30,20,15,pack_8,0
8,240,0,0,30,20,15,pack_9,0
9,0,20,0,30,20,15,pack_10,0


In [99]:
df_output.to_csv("Sample_Output_v1.2.csv")

In [111]:
df = pd.DataFrame()
df['Pack'] = ['box1', 'box2']

df['X'] = [0, 100]
df['Y'] = [0, 0]
df['Z'] = [0, 0]

df['L'] = [100, 60]
df['W'] = [40, 60]
df['H']  = [60, 60]

In [112]:
df

,Pack,X,Y,Z,L,W,H
0,box1,0,0,0,100,40,60
1,box2,100,0,0,60,60,60


In [113]:
df.to_csv("sample_output.csv", index = False)